O objetivo desse notebook é a construção de um modelo capaz de predizer o valor do IBU (*International Bitterness Units*), que mensura o armagor, para um conjunto de cervejas.

Os dados utilizados no treinamento foram coletados da [PUNK API](https://punkapi.com/documentation/v2) e posteriormente tratados e armazenados em buckets S3 na AWS.

Para utilizá-los nesse notebook, será feito uma query SQL via Athena na tabela Glue criada na AWS, que retornará os dados coletados em um certo período de tempo da API.

# Bibliotecas

In [166]:
import os
import dotenv
import time

import boto3
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
from  xgboost import XGBRegressor

# Leitura dos dados

In [107]:
dotenv.load_dotenv(".env")

S3_OUTPUT = "s3://punkapi-data-from-glue"
AWS_ACCESS_KEY_ID = os.environ["AWS_ACCESS_KEY_ID"]
AWS_SECRET_ACCESS_KEY = os.environ["AWS_SECRET_ACCESS_KEY"]
REGION_NAME = os.environ["REGION_NAME"]


class AthenaQuery:
    def __init__(self, query_string, database):
        self.database = database
        self.query_string = query_string

    def start_query(self):
        try:
            self.client = boto3.client(
                "athena",
                region_name=REGION_NAME,
                aws_access_key_id=AWS_ACCESS_KEY_ID,
                aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
            )
            query = self.client.start_query_execution(
                QueryString=self.query_string,
                QueryExecutionContext={"Database": self.database},
                ResultConfiguration={
                    "OutputLocation": S3_OUTPUT,
                },
            )
            self.current_query_id = query["QueryExecutionId"]
            print("Execution ID: " + self.current_query_id)

        except Exception as e:
            print(e)
        else:
            return query

    def get_query_status(self, query):
        query_status = self.client.get_query_execution(
            QueryExecutionId=self.current_query_id
        )["QueryExecution"]["Status"]["State"]
        return query_status

    def run(self):
        query = self.start_query()
        try:
            query_status = self.get_query_status(query)
            while query_status in ("QUEUED", "RUNNING"):
                query_status = self.get_query_status(query)
                print(query_status)
                if query_status in ("FAILED", "CANCELLED"):
                    raise Exception(
                        f'Athena query with the string "{self.query_string}" failed or was cancelled'
                    )
                time.sleep(10)
            print(f'Query "{self.query_string}" finished.')

            results = self.client.get_query_results(QueryExecutionId=query["QueryExecutionId"])

            return results

        except Exception as e:
            print(e)


# Colunas: ["abv", "ebc", "ibu", "id", "name", "ph", "srm", "target_fg", "target_og"]

query_string = (
    "SELECT TRY_CAST(abv as VARCHAR) as abv, "
    "TRY_CAST(ebc as VARCHAR) as ebc, "
    "TRY_CAST(ibu as VARCHAR) as ibu, "
    "TRY_CAST(id as VARCHAR) as id, "
    "TRY_CAST(name as VARCHAR) as name, "
    "TRY_CAST(ph as VARCHAR) as ph, "
    "TRY_CAST(srm as VARCHAR) as srm, "
    "TRY_CAST(target_fg as VARCHAR) as target_fg, "
    "TRY_CAST(target_og as VARCHAR) as target_og "
    "FROM cleaned_data_from_punkapi"
)
athena_query = AthenaQuery(query_string=query_string, database="punk_api_database")
results = athena_query.run()


Execution ID: eded1780-e617-45dd-8961-e1c832ab32c5
RUNNING
SUCCEEDED
Query "SELECT TRY_CAST(abv as VARCHAR) as abv, TRY_CAST(ebc as VARCHAR) as ebc, TRY_CAST(ibu as VARCHAR) as ibu, TRY_CAST(id as VARCHAR) as id, TRY_CAST(name as VARCHAR) as name, TRY_CAST(ph as VARCHAR) as ph, TRY_CAST(srm as VARCHAR) as srm, TRY_CAST(target_fg as VARCHAR) as target_fg, TRY_CAST(target_og as VARCHAR) as target_og FROM cleaned_data_from_punkapi" finished.


In [108]:
results

{'UpdateCount': 0,
 'ResultSet': {'Rows': [{'Data': [{'VarCharValue': 'abv'},
     {'VarCharValue': 'ebc'},
     {'VarCharValue': 'ibu'},
     {'VarCharValue': 'id'},
     {'VarCharValue': 'name'},
     {'VarCharValue': 'ph'},
     {'VarCharValue': 'srm'},
     {'VarCharValue': 'target_fg'},
     {'VarCharValue': 'target_og'}]},
   {'Data': [{'VarCharValue': '6.1'},
     {'VarCharValue': '219.0'},
     {'VarCharValue': '45'},
     {'VarCharValue': '16'},
     {'VarCharValue': 'Libertine Porter'},
     {'VarCharValue': '4.4'},
     {'VarCharValue': '109.5'},
     {'VarCharValue': '1020'},
     {'VarCharValue': '1067.0'}]},
   {'Data': [{'VarCharValue': '3.2'},
     {'VarCharValue': '7.0'},
     {'VarCharValue': '8'},
     {'VarCharValue': '240'},
     {'VarCharValue': 'Blitz Saison'},
     {'VarCharValue': '3.2'},
     {'VarCharValue': '14.0'},
     {'VarCharValue': '1007'},
     {'VarCharValue': '1040.0'}]},
   {'Data': [{'VarCharValue': '6.9'},
     {'VarCharValue': '15.0'},
     {'Va

In [101]:
def convert_athena_query_results_to_dataframe(results):
    data = results["ResultSet"]
    columns_names = [r["VarCharValue"] for r in data["Rows"][0]["Data"]]
    
    rows = [r["Data"] for r in data["Rows"][1:]]
    
    data_rows = [{columns_names[ix]: row[ix].get("VarCharValue", "") for ix in range(len(columns_names))} for row in rows]
    
    return pd.DataFrame(data_rows)

In [109]:
df = convert_athena_query_results_to_dataframe(results)

In [112]:
df

,abv,ebc,ibu,id,name,ph,srm,target_fg,target_og
0,6.1,219.0,45,16,Libertine Porter,4.4,109.5,1020,1067.0
1,3.2,7.0,8,240,Blitz Saison,3.2,14.0,1007,1040.0
2,6.9,15.0,35,256,Small Batch: Vermont IPA,4.2,7.62,1012,1066.0
3,9.7,30.0,100,170,Bashah (w/ Stone Brewing Co),4.4,15.3,1015,1088.0
4,15.2,158.0,20,96,Lumberjack Stout,4.4,79.0,1020,1110.0
...,...,...,...,...,...,...,...,...,...
185,16.1,400.0,85,177,Dog D,4.3,200.0,1015,1125.0
186,5.2,8.0,22,272,Small Batch: Spelt & Honey Saison,4.1,4.0,1008,1046.0
187,7.2,150.0,50,167,Libertine Black Ale,4.4,76.0,1013,1068.0
188,6.7,15.0,40,116,,,,,


# Pré-processamento do dado

Informações sobre o dado:

* ABV (Alcohol By Volume) - Quantidade de álcool presente na cerveja.
* EBC (European Brewery Convention) - Medida técnica europeia para a cor da cerveja.
* IBU (International Bitterness Units) - Índice que mede o amargor da cerveja.
* pH - Escala para determinar acidez/alcalinidade de uma solução.
* SRM (Standard Reference Method) - Sistema para medição de cor da cerveja usualmente aplicado nos Estados Unidos.
* Target FG (Final Gravity) - A gravidade final (FG) é a quantidade de açúcar que sobra quando a fermentação termina.
* Target OG (Original Gravity) - A gravidade original (OG) mede quanto açúcar está presente no mosto antes de ser fermentado. 

## Remoção de colunas

Duas colunas serão removidas do dado coletado: "id" e "name". A primeira, por se tratar apenas do identificador único de cada cerveja, não representa uma característica útil para determinar os IBU's. A segunda, apesar de poder ser transformada para um conjunto de valores numéricos via vetorização, será removida para simplificar o processo de treinamento.

In [115]:
INTEREST_COLUMNS = ["abv", "ebc", "ibu", "ph", "srm", "target_fg", "target_og"]
df_cleaned = df[INTEREST_COLUMNS]

In [116]:
df_cleaned

,abv,ebc,ibu,ph,srm,target_fg,target_og
0,6.1,219.0,45,4.4,109.5,1020,1067.0
1,3.2,7.0,8,3.2,14.0,1007,1040.0
2,6.9,15.0,35,4.2,7.62,1012,1066.0
3,9.7,30.0,100,4.4,15.3,1015,1088.0
4,15.2,158.0,20,4.4,79.0,1020,1110.0
...,...,...,...,...,...,...,...
185,16.1,400.0,85,4.3,200.0,1015,1125.0
186,5.2,8.0,22,4.1,4.0,1008,1046.0
187,7.2,150.0,50,4.4,76.0,1013,1068.0
188,6.7,15.0,40,,,,


## Ajuste dos tipos dos valores

Para evitar exceções na coleta dos dados utilizando a query via Athena, todos os valores foram transformados para string. Porém, é necessário que eles voltem a ter seus tipos originais para a realização do treinamento.

In [121]:
df_cleaned.dtypes

abv          object
ebc          object
ibu          object
ph           object
srm          object
target_fg    object
target_og    object
dtype: object

In [130]:
df_cleaned = df_cleaned.apply(pd.to_numeric)

In [131]:
df_cleaned.dtypes

abv          float64
ebc          float64
ibu            int64
ph           float64
srm          float64
target_fg    float64
target_og    float64
dtype: object

## Remover linhas com NaN's

In [132]:
df_cleaned.dropna(inplace=True)

## Feature scaling

Para auxiliar o treinamento e melhorar a performance do modelo, é importante que a escala dos valores presentes no dado de treinamento esteja contida dentro de determinada faixa. Dessa forma, será usado uma normalização min-max que manterá os valores entre 0 e 1.

In [150]:
scaler = MinMaxScaler()
scaled_data = df_cleaned.copy()
scaled_data[INTEREST_COLUMNS] = scaler.fit_transform(df_cleaned)

In [151]:
scaled_data

,abv,ebc,ibu,ph,srm,target_fg,target_og
0,0.138272,0.365000,0.041475,0.055696,0.359016,0.906667,0.927826
1,0.066667,0.011667,0.007373,0.040506,0.045902,0.895111,0.904348
2,0.158025,0.025000,0.032258,0.053165,0.024984,0.899556,0.926957
3,0.227160,0.050000,0.092166,0.055696,0.050164,0.902222,0.946087
4,0.362963,0.263333,0.018433,0.055696,0.259016,0.906667,0.965217
...,...,...,...,...,...,...,...
184,0.303704,0.185000,0.046083,0.055696,0.181967,0.906667,0.953043
185,0.385185,0.666667,0.078341,0.054430,0.655738,0.902222,0.978261
186,0.116049,0.013333,0.020276,0.051899,0.013115,0.896000,0.909565
187,0.165432,0.250000,0.046083,0.055696,0.249180,0.900444,0.928696


# Treinamento dos modelos

Para realizar a tarefa de predizer dos valores de IBU, serão testados dois modelos de regressão: um mais simples (uma regressão linear tradicional com mínimos quadrados) e outro mais elaborado (XGBoost).

In [134]:
X_COLUMNS = ["abv", "ebc", "ph", "srm", "target_fg", "target_og"]
Y_COLUMN = "ibu"

In [152]:
X, y = scaled_data[X_COLUMNS], scaled_data[Y_COLUMN]
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.1)

## Regressão linear

In [154]:
reg = LinearRegression().fit(X_train, y_train)
y_pred = reg.predict(X_test)

In [157]:
print(f"R2 (Reg. Linear): {r2_score(y_test, y_pred)}")

R2 (Reg. Linear): 0.9194819233255318


## XGBoost

In [167]:
xgb = XGBRegressor()
xgb.fit(X_train, y_train)
y_pred = xgb.predict(X_test)

R2 (XGBoost): 0.9921747939142996


In [168]:
print(f"R2 (XGBoost): {r2_score(y_test, y_pred)}")

R2 (XGBoost): 0.9921747939142996


# Treinando/testando modelos com um conjunto de dados maior

In [26]:
for page in range(1,11):
    print(page)
    res = requests.get(f"https://api.punkapi.com/v2/beers/?per_page=80&page={page}")
    data = res.json()
    df.extend(data)

1
2
3
4
5
6
7
8
9
10


In [29]:
df_ = pd.DataFrame(df)

In [31]:
df[0]

{'id': 1,
 'name': 'Buzz',
 'tagline': 'A Real Bitter Experience.',
 'first_brewed': '09/2007',
 'description': 'A light, crisp and bitter IPA brewed with English and American hops. A small batch brewed only once.',
 'image_url': 'https://images.punkapi.com/v2/keg.png',
 'abv': 4.5,
 'ibu': 60,
 'target_fg': 1010,
 'target_og': 1044,
 'ebc': 20,
 'srm': 10,
 'ph': 4.4,
 'attenuation_level': 75,
 'volume': {'value': 20, 'unit': 'litres'},
 'boil_volume': {'value': 25, 'unit': 'litres'},
 'method': {'mash_temp': [{'temp': {'value': 64, 'unit': 'celsius'},
    'duration': 75}],
  'fermentation': {'temp': {'value': 19, 'unit': 'celsius'}},
  'twist': None},
 'ingredients': {'malt': [{'name': 'Maris Otter Extra Pale',
    'amount': {'value': 3.3, 'unit': 'kilograms'}},
   {'name': 'Caramalt', 'amount': {'value': 0.2, 'unit': 'kilograms'}},
   {'name': 'Munich', 'amount': {'value': 0.4, 'unit': 'kilograms'}}],
  'hops': [{'name': 'Fuggles',
    'amount': {'value': 25, 'unit': 'grams'},
    '